In [1]:
import benchmarking

MODEL_FOLDER = "../models/v2/"
DATA_FOLDER = "../data/samples/" 
RESULT_FOLDER = "../results/" 

SAMPLES_G = ["SGSH"] #, "SFSH", "SPSH", "SSSH"]
SAMPLES_P = ["SPSH"]
SAMPLES_S = ["SSSH"]

RETRIEVAL_CONFIGS = [
    {"k": 5, "threshold": t}
    for t in [0.672, 0.674, 0.676, 0.678]
]

RERANKER_CONFIGS = [0.3, 0.4, 0.5, 0.6, 0.7]

In [ ]:
MODELS = [f"BEmargin_{7}_0"]

for MODEL in MODELS:
    benchmarking.BenchmarkRunner(
            MODEL_FOLDER, [MODEL],
            DATA_FOLDER, {'G': SAMPLES_G, 'P': SAMPLES_P, 'S': SAMPLES_S},
            RESULT_FOLDER,
            None, RETRIEVAL_CONFIGS
        ).benchmark("benchmark-BE-v2-(m7, t672-678).json")

In [2]:
MODELS = ["CESweight_0_5"] + [f"CESweight_{k}_0" for k in [1,2,5,10]]

for MODEL in MODELS:
    benchmarking.BenchmarkRunner(
            MODEL_FOLDER, [None, None, MODEL],
            DATA_FOLDER, {'G': SAMPLES_G, 'P': SAMPLES_P, 'S': SAMPLES_S},
            RESULT_FOLDER,
            None, RETRIEVAL_CONFIGS, RERANKER_CONFIGS
        ).benchmark("benchmark-CES-v2-(w0.5-10).json")

Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:48<00:00, 20.44it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:45<00:00, 21.82it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:51<00:00, 19.55it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.44it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.41it/s]


In [5]:
MODELS = ["CEPweight_0_5"] + [f"CEPweight_{k}_0" for k in [1,2,5,10]]

for MODEL in MODELS:
    benchmarking.BenchmarkRunner(
            MODEL_FOLDER, [None, MODEL, None],
            DATA_FOLDER, {'G': SAMPLES_G, 'P': SAMPLES_P, 'S': SAMPLES_S},
            RESULT_FOLDER,
            None, RETRIEVAL_CONFIGS, RERANKER_CONFIGS
        ).benchmark("benchmark-CEP-v2-(w0.5-10, t3-7).json")

Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:45<00:00, 22.00it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:44<00:00, 22.24it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:45<00:00, 22.17it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:45<00:00, 22.20it/s]


Starting Benchmarking in Single Model Mode.
Starting Reranker Benchmarking.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:46<00:00, 21.56it/s]
